In [2]:
import requests
import json
import time

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
pinata_server = 'https://hub.pinata.cloud'
dbStats = requests.get(f'{pinata_server}/v1/info?dbstats=1')

In [11]:
dbStats.json()['dbStats']['numFidEvents']

608566

In [6]:
# base_url = f"{pinata_server}/v1/userDataByFid"
# params = {"fid": 2, "pageSize":1}  # Initial parameters (adjust as needed)
# next_page_token = None

# while True:
#     if next_page_token:
#         params["pageToken"] = next_page_token

#     response = requests.get(base_url, params=params)

#     # Error handling (check for 200 OK status)
#     response.raise_for_status()

#     data = response.json()
#     # print(data)
#     # Process the current page's results here (e.g., print cast details)
#     for cast in data.get("messages", []):
#         print(cast)

#     next_page_token = data.get("nextPageToken")
#     if not next_page_token:
#         break

In [7]:
def fetch_all_data(base_url, params, data_key, max_requests_per_minute=185):
    """Fetches all pages of data from a paginated API.

    Args:
        base_url: The base URL of the API endpoint.
        params: A dictionary of initial query parameters.
        data_key: The key in the response JSON that contains the data array.

    Returns:
        A list containing all the data objects from all pages.
    """
    all_data = []
    next_page_token = None
    request_count = 0
    start_time = time.monotonic()

    while True:
        if next_page_token:
            params["pageToken"] = next_page_token

        response = requests.get(base_url, params=params)
        response.raise_for_status()  # Raise an exception for bad responses

        data = response.json()
        all_data.extend(data.get(data_key, []))

        next_page_token = data.get("nextPageToken")

        request_count += 1
        elapsed_time = time.monotonic() - start_time
        if request_count >= max_requests_per_minute:
            time_to_sleep = 60 - elapsed_time  # Seconds remaining in the minute
            if time_to_sleep > 0:
                time.sleep(time_to_sleep)
            request_count = 0
            start_time = time.monotonic()

        if not next_page_token:
            break

    return all_data



In [9]:
# Example usage with Farcaster Hub API (adjust parameters as needed)
all_gets = []
for userFid in range(1, dbStats.json()['dbStats']['numFidEvents']):
  base_url = f"{pinata_server}/v1/userDataByFid"
  params = {"fid": userFid, "pageSize": 100}  # Larger page size for efficiency
  data_key = "messages"

  all_messages = fetch_all_data(base_url, params, data_key)
  # all_gets.extend(all_messages)
  # Save aggregated data to a JSON file
  with open(f'/content/drive/My Drive/farcaster_hub/userData/{userFid}.json', "w", encoding='utf-8') as outfile:
      json.dump(all_messages, outfile, ensure_ascii=False, indent=4)

HTTPError: 524 Server Error:  for url: https://hub.pinata.cloud/v1/userDataByFid?fid=2489&pageSize=100

In [ ]:
# for userFid in range(1, dbStats.json()['dbStats']['numFidEvents']):
#     userData = requests.get(f'{_server}/v1/userDataByFid?fid={userFid}')
#     print(userFid)
#     try:
#         with open(f'/content/drive/My Drive/farcaster_hub/users/{userFid}.json', 'w', encoding='utf-8') as f:
#             json.dump(userData.json(), f, ensure_ascii=False, indent=4)
#     except:
#         continue

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65


KeyboardInterrupt: 

In [ ]:
for userFid in range(1, dbStats.json()['dbStats']['numFidEvents']):
    userData = requests.get(f'{_server}/v1/userDataByFid?fid={userFid}')
    print(userFid)
    try:
        with open(f'/content/drive/My Drive/farcaster_hub/users/{userFid}.json', 'w', encoding='utf-8') as f:
            json.dump(userData.json(), f, ensure_ascii=False, indent=4)
    except:
        continue